## TEST

In [1]:
from Levenshtein import distance
import tensorflow as tf
import os
from inference_smartapp import handwriting_model

MAFIA_BOSS_LINES_NEW_DS_adam
Loading pre-trained model and weights...


TypeError: __init__() takes from 2 to 4 positional arguments but 5 were given

In [ ]:
dataset_path = 'dataset/transfer_dataset/'
val_dataset_path = os.path.join(dataset_path, 'val')

In [ ]:
handwriting_model.model.summary()

In [ ]:
val_df_list = os.listdir(val_dataset_path)
val_df_jpg_list = [val_df_list[i] for i in range(len(val_df_list)) if val_df_list[i].endswith('.jpg')]
val_df_jpg_list = [os.path.join(val_dataset_path, val_df_jpg_list[i]) for i in range(len(val_df_jpg_list)) ]

In [ ]:
scores = []
pred_real = []
for image_path in val_df_jpg_list:
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 1)
    prediction = handwriting_model.inference(image)
    with open(image_path.replace('.jpg', '.txt'), 'r') as f:
        label = f.read().replace("|", " ")
    scores.append(distance(prediction, label))
    pred_real.append((prediction, label))
    #print(f'Prediction: {prediction} Label: {label} Distance: {scores[-1]}')


In [ ]:
print(f'Mean distance: {round(sum(scores) / len(scores),2)}')

In [ ]:
print("(Prediction,Real)")
for pred_r in pred_real:
    print(pred_r)

In [ ]:
# Test single files
image_path = "dataset/transfer_dataset/AD__0001.png"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image, 1)
prediction = handwriting_model.inference(image)
print(prediction)